In [1]:
import numpy as np

In [2]:
a = np.arange(24)

In [3]:
a =a.reshape(2,3,4)

In [4]:
a

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [5]:
a_del = np.delete(a, -1, 2)
a_del

array([[[ 0,  1,  2],
        [ 4,  5,  6],
        [ 8,  9, 10]],

       [[12, 13, 14],
        [16, 17, 18],
        [20, 21, 22]]])

In [6]:
a[..., 1]

array([[ 1,  5,  9],
       [13, 17, 21]])

In [7]:
a

array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]],

       [[12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]]])

In [8]:
b = np.array([[0, 1], [0,0]])
np.tile(b, (2, 2))

array([[0, 1, 0, 1],
       [0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 0]])

In [9]:
b = b[None, None]

In [10]:
b.shape

(1, 1, 2, 2)

In [11]:
c = np.array([[1],[2],[3]])
d = np.array([[2],[3],[4]])
e = np.dstack((c,d))
e.shape

(3, 1, 2)

## RGB маска

In [12]:
def get_bayer_masks(n_rows, n_cols):
  rows = n_rows // 2 + n_rows % 2
  cols = n_cols // 2 + n_cols % 2
  red = np.array([[0, 1], [0,0]], dtype=bool)
  red = np.tile(red, (rows, cols))
  green = np.array([[1, 0], [0,1]], dtype=bool)
  green = np.tile(green, (rows, cols))
  blue = np.array([[0, 0], [1,0]], dtype=bool)
  blue = np.tile(blue, (rows, cols))
  if n_rows % 2 != 0:   #suboptimal way of working with odd matrices
    red = np.delete(red, -1, 0)
    green = np.delete(green, -1, 0)
    blue = np.delete(blue, -1, 0)
  if n_cols % 2 != 0:
    red = np.delete(red, -1, 1)
    green = np.delete(green, -1, 1)
    blue = np.delete(blue, -1, 1)
  return np.dstack((red, green, blue))

In [13]:
bayer_mask = get_bayer_masks(3, 3)

In [14]:
bayer_mask[..., ]

array([[[False,  True, False],
        [ True, False, False],
        [False,  True, False]],

       [[False, False,  True],
        [False,  True, False],
        [False, False,  True]],

       [[False,  True, False],
        [ True, False, False],
        [False,  True, False]]])

In [15]:
from numpy import *
gt_masks = zeros((3, 3, 3), 'bool')
gt_masks[..., 0] = array([[0, 1, 0],
                          [0, 0, 0],
                          [0, 1, 0]])
gt_masks[..., 1] = array([[1, 0, 1],
                          [0, 1, 0],
                          [1, 0, 1]])
gt_masks[..., 2] = array([[0, 0, 0],
                          [1, 0, 1],
                          [0, 0, 0]])
gt_masks

array([[[False,  True, False],
        [ True, False, False],
        [False,  True, False]],

       [[False, False,  True],
        [False,  True, False],
        [False, False,  True]],

       [[False,  True, False],
        [ True, False, False],
        [False,  True, False]]])

##ТРехканальное изображение

In [16]:
raw_im = np.arange(32).reshape(4,8)
raw_im

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31]])

In [17]:
def get_colored_img(raw_img):
  mask = get_bayer_masks(raw_img.shape[0], raw_img.shape[1])
  raw_image = np.dstack((raw_img, raw_img, raw_img))
  return mask * raw_image

In [18]:
rgb = get_colored_img(raw_im)
rgb[:,:,2]

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  0, 10,  0, 12,  0, 14,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [24,  0, 26,  0, 28,  0, 30,  0]])

##Билинейная интерполяция

In [19]:
x = np.array([[1., 2.], [np.nan, 3.], [np.nan, np.nan]])
np.mean(x[~np.isnan(x)])

2.0

In [20]:
mask = get_bayer_masks(raw_im.shape[0], raw_im.shape[1])
print(mask[0:3, 0:3, 0])
print(raw_im * mask[..., 0])
print(raw_im[0:3, 0:3][mask[0:3, 0:3, 0]])
print(np.mean(raw_im[0:3, 0:3][mask[0:3, 0:3, 0]]))

[[False  True False]
 [False False False]
 [False  True False]]
[[ 0  1  0  3  0  5  0  7]
 [ 0  0  0  0  0  0  0  0]
 [ 0 17  0 19  0 21  0 23]
 [ 0  0  0  0  0  0  0  0]]
[ 1 17]
9.0


In [21]:
np.uint8(3.65)

3

In [22]:
from scipy.signal import convolve2d

In [23]:
def bilinear_interpolation(colored_img):
  mask = get_bayer_masks(colored_img.shape[0], colored_img.shape[1])
  interpolated_image= colored_img.copy()
  kernel_rb = np.array([[0.25, 0.5, 0.25], [0.5, 1, 0.5], [0.25, 0.5, 0.25]])
  kernel_g = np.array([[0, 0.25, 0], [0.25, 1, 0.25], [0, 0.25, 0]])
  interpolated_image[..., 0] = np.uint8(convolve2d(colored_img[..., 0], kernel_rb, mode='same', boundary='fill', fillvalue=0))
  interpolated_image[..., 1] = np.uint8(convolve2d(colored_img[..., 1], kernel_g, mode='same', boundary='fill', fillvalue=0))
  interpolated_image[..., 2] = np.uint8(convolve2d(colored_img[..., 2], kernel_rb, mode='same', boundary='fill', fillvalue=0))
  return interpolated_image

In [24]:
interpolated = bilinear_interpolation(rgb)
interpolated[..., 2]

array([[ 4,  4,  5,  5,  6,  6,  7,  3],
       [ 8,  9, 10, 11, 12, 13, 14,  7],
       [16, 17, 18, 19, 20, 21, 22, 11],
       [24, 25, 26, 27, 28, 29, 30, 15]])

In [25]:
def old_bilinear_interpolation(colored_img):
   print(colored_img.shape)
   mask = get_bayer_masks(colored_img.shape[0], colored_img.shape[1])
   interpolated_image = colored_img.copy()
   for k in range(colored_img.shape[2]):
     for j in range(1, colored_img.shape[1]-1):
       for i in range(1, colored_img.shape[0]-1):
         if mask[i,j,k] == True:
           continue
         else:
           interpolated_image[i,j,k] = np.uint8(np.mean(colored_img[i-1:i+2,     j-1:j+2, k][mask[i-1:i+2, j-1:j+2, k]]))
   return interpolated_image

In [26]:
old_interpolated = old_bilinear_interpolation(rgb)
old_interpolated[..., 2]

(4, 8, 3)


array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 8,  9, 10, 11, 12, 13, 14,  0],
       [ 0, 17, 18, 19, 20, 21, 22,  0],
       [24,  0, 26,  0, 28,  0, 30,  0]])

In [27]:
s = np.array([[0.25, 0.5, 0.25], [0.5, 1, 0.5], [0.25, 0.5, 0.25]])
g = np.array([[4, 0, 4], [0, 0, 0], [4, 0, 4]])

res = convolve2d(g, s, mode='same', boundary='fill', fillvalue=0)
res

array([[4., 4., 4.],
       [4., 4., 4.],
       [4., 4., 4.]])

##Улучшенная линейная интерполяция

In [46]:
def improved_interpolation(raw_img):
  alpha = 0.5
  beta = 0.625
  gamma = 0.75
  colored_img = get_colored_img(raw_img)
  kernel_g_rb = np.array([[0,0,-0.25,0,0], [0,0,0,0,0], [-0.25,0,1,0,-0.25], [0,0,0,0,0], [0,0,-0.25,0,0]])
  improved = np.float64(bilinear_interpolation(colored_img))

  #Блок с поправкой green color в R+B ячейках
  delta_gr = convolve2d(colored_img[..., 0], kernel_g_rb, mode='same', boundary='fill', fillvalue=0)
  delta_gb = convolve2d(colored_img[..., 2], kernel_g_rb, mode='same', boundary='fill', fillvalue=0)
  improved[..., 1] += alpha * (delta_gr + delta_gb) #с green закончили

  #Блок с корректировкой красного и голубого цветов, но в ячейках зеленого цвета
  kernel_rb_g_horizontal = np.array([[0,0,0.1,0,0], [0,-0.2,0,-0.2,0], [-0.2,0,1,0,-0.2], [0,-0.2,0,-0.2,0], [0,0,0.1,0,0]])
  kernel_rb_g_vertical = np.array([[0,0,-0.2,0,0], [0,-0.2,0,-0.2,0], [0.1,0,1,0,0.1], [0,-0.2,0,-0.2,0], [0,0,-0.2,0,0]])
  delta_rg_horizontal = convolve2d(colored_img[..., 1], kernel_rb_g_horizontal, mode='same', boundary='fill', fillvalue=0)
  delta_rg_horizontal[1::2] = 0
  delta_rg_vertical = convolve2d(colored_img[..., 1], kernel_rb_g_vertical, mode='same', boundary='fill', fillvalue=0)
  delta_rg_vertical[::2] = 0
  improved[..., 0] += beta * (delta_rg_horizontal + delta_rg_vertical) #поправка red в точках green


  delta_bg_horizontal = convolve2d(colored_img[..., 1], kernel_rb_g_horizontal, mode='same', boundary='fill', fillvalue=0)
  delta_bg_horizontal[::2] = 0
  delta_bg_vertical = convolve2d(colored_img[..., 1], kernel_rb_g_vertical, mode='same', boundary='fill', fillvalue=0)
  delta_bg_vertical[1::2] = 0
  improved[..., 2] += beta * (delta_bg_horizontal + delta_bg_vertical) #поправка blue в точках green

  #поправка RB в BR
  kernel_rb_br = np.array([[0,0,-0.25,0,0], [0,0,0,0,0], [-0.25,0,1,0,-0.25], [0,0,0,0,0], [0,0,-0.25,0,0]])
  delta_r_b = convolve2d(colored_img[..., 2], kernel_rb_br, mode='same', boundary='fill', fillvalue=0)
  improved[...,0] += gamma * delta_r_b #закончили с красным

  delta_b_r = convolve2d(colored_img[..., 0], kernel_rb_br, mode='same', boundary='fill', fillvalue=0)
  improved[...,2] += gamma * delta_b_r #закончили с синим
  return np.uint8(np.clip(improved, 0, 255))

In [49]:
arr = np.arange(343).reshape((7,7,7))
arr = np.uint8(np.clip(arr, 0, 255))
arr

array([[[  0,   1,   2,   3,   4,   5,   6],
        [  7,   8,   9,  10,  11,  12,  13],
        [ 14,  15,  16,  17,  18,  19,  20],
        [ 21,  22,  23,  24,  25,  26,  27],
        [ 28,  29,  30,  31,  32,  33,  34],
        [ 35,  36,  37,  38,  39,  40,  41],
        [ 42,  43,  44,  45,  46,  47,  48]],

       [[ 49,  50,  51,  52,  53,  54,  55],
        [ 56,  57,  58,  59,  60,  61,  62],
        [ 63,  64,  65,  66,  67,  68,  69],
        [ 70,  71,  72,  73,  74,  75,  76],
        [ 77,  78,  79,  80,  81,  82,  83],
        [ 84,  85,  86,  87,  88,  89,  90],
        [ 91,  92,  93,  94,  95,  96,  97]],

       [[ 98,  99, 100, 101, 102, 103, 104],
        [105, 106, 107, 108, 109, 110, 111],
        [112, 113, 114, 115, 116, 117, 118],
        [119, 120, 121, 122, 123, 124, 125],
        [126, 127, 128, 129, 130, 131, 132],
        [133, 134, 135, 136, 137, 138, 139],
        [140, 141, 142, 143, 144, 145, 146]],

       [[147, 148, 149, 150, 151, 152, 153],
    

In [55]:
raw_img = array([[8, 5, 3, 7, 1, 3],
                         [5, 2, 6, 8, 8, 1],
                         [9, 9, 8, 1, 6, 4],
                         [9, 4, 2, 3, 6, 8],
                         [5, 4, 3, 2, 8, 7],
                         [7, 3, 3, 6, 9, 3]], dtype='uint8')


AttributeError: ignored

##Метрика PSNR

In [52]:
from skimage.metrics import mean_squared_error

In [56]:
def compute_psnr(img_pred, img_gt):
  mse = np.sum((np.float64(img_pred) - np.float64(img_gt)) ** 2) / np.prod(img_pred.shape)
  if mse == 0:
    raise ValueError("MSE is zero!")
  return 10 * np.log10((np.max(np.float64(img_gt)) ** 2 )/ mse)